## Homework

> Note: sometimes your answer doesn't match one of the options exactly. That's fine. Select the option that's closest to your solution.

**Dataset**

In this homework, we will use the lead scoring dataset Bank Marketing dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv).

Or you can do it with ```wget```:
```
wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
```
In this dataset our desired target for classification task will be the ```converted``` variable - has the client signed up to the platform or not.

**Data preparation**

* Check if the missing values are presented in the features.
* If there are missing values:
    * For caterogiral features, replace them with 'NA'
    * For numerical features, replace with with 0.0

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
# !wget $data

In [4]:
df = pd.read_csv('course_lead_scoring.csv')
df

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1
...,...,...,...,...,...,...,...,...,...
1457,referral,manufacturing,1,NaN,self_employed,north_america,4,0.53,1
1458,referral,technology,3,65259.0,student,europe,2,0.24,1
1459,paid_ads,technology,1,45688.0,student,north_america,3,0.02,1
1460,referral,NaN,5,71016.0,self_employed,north_america,0,0.25,1


In [5]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [6]:
# Do not include the target column
categorical = ['industry', 'employment_status', 'location']
numerical = ['number_of_courses_viewed', 'annual_income', 'interaction_count', 'lead_score']

Check for nul values

In [7]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

Set nulls in categorical columns to NA

In [8]:
for c in categorical:
    df[c] = df[c].fillna('NA')

df.isnull().sum()

lead_source                 128
industry                      0
number_of_courses_viewed      0
annual_income               181
employment_status             0
location                      0
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

annual_income is the only numerical column with null values.

In [9]:
df.annual_income = df.annual_income.fillna(0.0)

In [10]:
df.isnull().sum()

lead_source                 128
industry                      0
number_of_courses_viewed      0
annual_income                 0
employment_status             0
location                      0
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [11]:
df.converted.unique()

array([1, 0])

In [12]:
df.converted.mean()

np.float64(0.6190150478796169)

The target column ```converted``` has a coverted == True percentage of aprox 62%

## Question 1

What is the most frequent observation (mode) for the column industry?

* ```NA```
* ```technology```
* ```healthcare```
* ```retail```      <- highest at 203


In [13]:
df.industry.value_counts()

industry
retail           203
finance          200
other            198
healthcare       187
education        187
technology       179
manufacturing    174
NA               134
Name: count, dtype: int64

## Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

* ```interaction_count``` and ```lead_score```
* ```number_of_courses_viewed``` and ```lead_score```
* ```number_of_courses_viewed``` and ```interaction_count```
* ```annual_income``` and ```interaction_count``` <-- has a corralation of 1!!

Only consider the pairs above when answering this question.

In [14]:
df[numerical].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


## Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
* Make sure that the target value y is not in your dataframe.

In [15]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [16]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [17]:
len(df), len(df_full_train), len(df_train), len(df_test), len(df_val)

(1462, 1169, 876, 293, 293)

In [18]:
# reset the index so they are sequential
df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [19]:
# get converted 
y_train = df_train.converted.values
y_val= df_val.converted.values
y_test = df_test.converted.values   

In [20]:
# remove converted from data frames
del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 3

* Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
* Round the scores to 2 decimals using ```round(score, 2)```.

Which of these variables has the biggest mutual information score?

* ```industry```
* ```location```
* ```lead_source```. <- has a 0.03
* ```employment_status```

In [21]:
round(mutual_info_score(df_full_train.converted, df_full_train.industry), 2)

0.01

In [22]:
round(mutual_info_score(df_full_train.converted, df_full_train.location), 2)

0.0

In [23]:
round(mutual_info_score(df_full_train.converted, df_full_train.lead_score), 2)

/opt/conda/envs/ml-zoomcamp/lib/python3.11/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received binary values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


0.07

In [24]:
round(mutual_info_score(df_full_train.converted, df_full_train.employment_status), 2)

0.01

## Question 4

Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    * To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    * ```model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)```
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

* 0.64
* 0.74 <- closest
* 0.84
* 0.94

Use one-hot encoding to encode categorical variables

In [25]:
# convert training data frame to a list of dictionaries.
train_dict = df_train[categorical + numerical].to_dict(orient='records')

In [26]:
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)  # does the encoding

X_train = dv.transform(train_dict)
X_train[0]

array([5.8472e+04, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 1.0000e+00, 0.0000e+00, 5.0000e+00,
       3.0000e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00])

In [27]:
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'location=NA', 'location=africa', 'location=asia',
       'location=australia', 'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

Do the encoding for the val data frame.

In [28]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [29]:
dv.fit(val_dicts)  # does the encoding

X_val = dv.transform(val_dicts)

**Logistic Regression**

In [30]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [31]:
model.intercept_[0] # the bias

np.float64(-0.0866284554424958)

In [32]:
model.coef_ # the weights

array([[-1.72671104e-05, -1.66458788e-02,  3.68229530e-02,
         9.22443543e-04,  9.59431286e-03, -1.17322286e-01,
        -2.82403017e-02,  5.37452226e-02, -2.32546796e-02,
        -1.68524466e-02, -4.47556538e-03, -1.13881259e-02,
        -3.73374591e-02, -1.88250997e-02,  2.95032896e-01,
         5.25590013e-02,  4.00110626e-03, -1.39640666e-02,
        -1.36215426e-02, -7.57345524e-03,  7.02627243e-03,
         4.65096224e-03, -3.90201047e-02, -2.81276272e-02,
         4.65862791e-01]])

In [33]:
y_pred = model.predict_proba(X_val)[:, 1]
y_pred

array([0.64125951, 0.7963558 , 0.51779835, 0.4683655 , 0.53885963,
       0.45622396, 0.86170091, 0.84152704, 0.83827261, 0.62451021,
       0.55487194, 0.75088184, 0.70858022, 0.75535319, 0.52096866,
       0.91116114, 0.5335346 , 0.41392616, 0.29803196, 0.85437275,
       0.77165843, 0.72682348, 0.43981214, 0.61974947, 0.39453329,
       0.74470707, 0.89705466, 0.32408153, 0.42632662, 0.97030346,
       0.9135542 , 0.36131924, 0.65922775, 0.89470789, 0.73939803,
       0.6448289 , 0.82470944, 0.84303751, 0.65658253, 0.32834269,
       0.80279398, 0.36599156, 0.96474677, 0.599004  , 0.51563057,
       0.53592413, 0.82023882, 0.76843457, 0.73300011, 0.67009312,
       0.46818753, 0.85802125, 0.56459508, 0.93175978, 0.62133458,
       0.62956739, 0.63225466, 0.30383051, 0.4694563 , 0.57431673,
       0.37932066, 0.60112216, 0.39655222, 0.6073638 , 0.84918485,
       0.74071904, 0.89181118, 0.71253579, 0.94922196, 0.88730229,
       0.76363586, 0.33666065, 0.59210408, 0.54639951, 0.63514

In [34]:
customers_converted = y_pred > 0.5

In [35]:
accuracy = (y_val == customers_converted).mean()
accuracy

np.float64(0.7030716723549488)

## Question 5

Let's find the least useful feature using the feature elimination technique.
* Train a model using the same features and parameters as in Q4 (without rounding).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

Which of following feature has the smallest difference?

* ```'industry'```
* ```'employment_status'```. <- smallest difference
* ```'lead_score'```

> Note: The difference doesn't have to be positive.

In [36]:
df_full_train_5, df_test_5 = train_test_split(df, test_size=0.2, random_state=42)
df_train_5, df_val_5 = train_test_split(df_full_train, test_size=0.25, random_state=42)

df_train_industry = df_train_5.copy()
df_train_employment_status = df_train_5.copy()
df_train_lead_score = df_train_5.copy()

del df_train_industry['industry']
del df_train_employment_status['employment_status']
del df_train_lead_score['lead_score']

y_train_industry = df_train_industry.converted.values
y_val_industry = df_train_industry.converted.values
y_test_industry = df_train_industry.converted.values 

y_train_employment_status = df_train_employment_status.converted.values
y_val_employment_status = df_train_employment_status.converted.values
y_test_employment_status = df_train_employment_status.converted.values 

y_train_lead_score = df_train_lead_score.converted.values
y_val_lead_score = df_train_lead_score.converted.values
y_test_lead_score = df_train_lead_score.converted.values

categorical_industry = [c for c in categorical if c != 'industry']
categorical_employment_status = [c for c in categorical if c != 'employment_status']
numerical_lead_score = [c for c in numerical if c != 'lead_score']

train_industry_dicts = df_train_industry[categorical_industry + numerical].to_dict(orient='records')
train_employment_status_dicts = df_train_employment_status[categorical_employment_status + numerical].to_dict(orient='records')
train_lead_score_dicts = df_train_lead_score[categorical + numerical_lead_score].to_dict(orient='records')

val_industry_dicts = df_val[categorical_industry + numerical].to_dict(orient='records')
val_employment_status_dicts = df_val[categorical_employment_status + numerical].to_dict(orient='records')
val_lead_score_dicts = df_val[categorical + numerical_lead_score].to_dict(orient='records')


In [37]:
dv_industry = DictVectorizer(sparse=False)
dv_industry.fit(train_industry_dicts)  
X_industry = dv_industry.transform(train_industry_dicts)
X_val_industry = dv_industry.transform(val_industry_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_industry, y_train_industry)
y_pred_industry = model.predict_proba(X_val_industry)[:, 1]
industry_customers_converted = y_pred_industry > 0.5
industry_accuracy = (y_val == industry_customers_converted).mean()
industry_accuracy, accuracy - industry_accuracy

(np.float64(0.7030716723549488), np.float64(0.0))

In [38]:
dv_employment_status = DictVectorizer(sparse=False)
dv_employment_status.fit(train_employment_status_dicts)  
X_employment_status = dv_employment_status.transform(train_industry_dicts)
X_val_employment_status = dv_employment_status.transform(val_employment_status_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_employment_status, y_train_employment_status)
y_pred_employment_status = model.predict_proba(X_val_employment_status)[:, 1]
employment_status_customers_converted = y_pred_employment_status > 0.5
employment_status_accuracy = (y_val == employment_status_customers_converted).mean()
employment_status_accuracy, accuracy - employment_status_accuracy

(np.float64(0.6928327645051194), np.float64(0.010238907849829393))

In [39]:
dv_lead_score = DictVectorizer(sparse=False)
dv_lead_score.fit(train_lead_score_dicts)  
X_lead_score = dv_lead_score.transform(train_lead_score_dicts)
X_val_lead_score = dv_lead_score.transform(val_lead_score_dicts)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_lead_score, y_train_lead_score)
y_pred_lead_score = model.predict_proba(X_val_lead_score)[:, 1]
employment_lead_score_customers_converted = y_pred_lead_score > 0.5
lead_score_accuracy = (y_val == employment_lead_score_customers_converted).mean()
lead_score_accuracy, accuracy - lead_score_accuracy

(np.float64(0.6996587030716723), np.float64(0.0034129692832765013))

In [40]:
employment_status_accuracy < lead_score_accuracy

np.True_

## Question 6

Now let's train a regularized logistic regression.
* Let's try the following values of the parameter ```C: [0.01, 0.1, 1, 10, 100]```.
* Train models using all the features as in Q4.
* Calculate the accuracy on the validation dataset and round it to 3 decimal digits.

Which of these ```C``` leads to the best accuracy on the validation set?

* 0.01
* 0.1
* 1 <-
* 10 
* 100 

> Note: If there are multiple options, select the smallest ```C```.

In [42]:
for c in [0.001, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    customers_converted = y_pred > 0.5
    accuracy = (y_val == customers_converted).mean()
    print(f"{c}\t{accuracy}")
    

0.001	0.5631399317406144
0.1	0.6996587030716723
1	0.7030716723549488
10	0.7030716723549488
100	0.7030716723549488
